<a href="https://colab.research.google.com/github/hanarayan/EPAM_PRACTICE/blob/main/PracticeEPAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install pyspark

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.


In [ ]:
from pyspark import SparkContext

# Check if a SparkContext already exists
try:
    sc = SparkContext.getOrCreate()
    print("Using existing SparkContext")
except ValueError:
    # If not, create a new one
    sc = SparkContext("local", "FlatMap Example")
    print("Created a new SparkContext")


Using existing SparkContext


In [ ]:
# Read the CSV file as an RDD
rdd = sc.textFile("/content/LinkedIn people profiles datasets.csv")


In [ ]:
# Dont consider the header
header = rdd.first()
rdd_no_header = rdd.filter(lambda line: line != header)
print(f"Total rows (Including header): {rdd.count()}")
print(f"Total rows (excluding header): {rdd_no_header.count()}")

Total rows (Including header): 1001
Total rows (excluding header): 1000


In [ ]:
# Split each row into columns
rdd_split = rdd_no_header.map(lambda line: line.split(","))

header_columns = header.split(",")

# Convert column names to lowercase and remove leading/trailing spaces for comparison
header_columns = [col.strip('""').strip().lower() for col in header_columns]

print("Available Columns:")
for column in header_columns:
    print(column)

country_code_index = header_columns.index("country_code")

# Extract country codes from the relevant column
country_codes_rdd = rdd_split.map(lambda row: row[country_code_index])

# Get distinct country codes
distinct_countries = (
    rdd_split.map(lambda row: row[country_code_index].strip('"').strip())  # Remove extra quotes
    .distinct()
    .collect()
)

#Print the distinct country codes
print("Distinct Country Codes:")
for country in distinct_countries:
    print(country)

In [ ]:
region_index = header_columns.index("region")
regions_rdd  = rdd_split.map(lambda row: (row[region_index].strip('"').strip(), 1))
region_counts = regions_rdd.reduceByKey(lambda a, b: a + b)
region_counts_result = region_counts.collect()
for region, count in region_counts_result:
    print(f"Region: {region}, Count: {count}")
